In [1]:
import os
import json
import cv2
import numpy as np

import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt

import pvlib
from datetime import datetime

# Load Data

In [2]:
marikina_gdf = gpd.read_file("marikina_polygon_bounds.geojson")
marikina_gdf.head()

,id,@id,admin_level,boundary,name,population,source:population,type,wikidata,wikipedia,...,name:tl,layer,tunnel,gns_classification,gns_uni,name:ja,name:ru,place,population:date,geometry
0,relation/146949,relation/146949,6,administrative,Marikina,450741,Philippines 2015 Census,boundary,Q17175,en:Marikina,...,None,None,None,None,None,None,None,None,None,"POLYGON ((121.09869 14.66569, 121.09839 14.664..."


In [3]:
gdf = gpd.read_file("solar_potential_flatsouth.geojson")
gdf.head()

,id,address,num_panels,panel_area,avg_hour_0,avg_hour_1,avg_hour_2,avg_hour_3,avg_hour_4,avg_hour_5,...,avg_month_5,avg_month_6,avg_month_7,avg_month_8,avg_month_9,avg_month_10,avg_month_11,avg_month_12,total_kwh,geometry
0,way/4392200,"J3MX+763, Marikina, 1800 Metro Manila, Philipp...",4.0,6.5472,0.0,0.0,0.0,0.0,0.0,0.000801,...,120.874923,110.667060,108.223943,100.522987,107.129981,85.413911,85.992027,89.878759,1.282380e+03,"POLYGON ((121.09815 14.63357, 121.09786 14.633..."
1,way/4947814,"1800 Riverbanks Ave, Marikina, 1800 Metro Mani...",3308.0,5414.5344,0.0,0.0,0.0,0.0,0.0,0.676771,...,97969.436917,91088.852380,88415.504533,79503.541618,81989.875234,63640.141696,61632.344911,63619.649196,9.787500e+05,"POLYGON ((121.08072 14.62996, 121.08078 14.629..."
2,way/4947816,"300 Riverbanks Ave, Sto. Nino, Marikina, 1800 ...",15753.0,25784.5104,0.0,0.0,0.0,0.0,0.0,6.866575,...,475842.932919,443030.099559,430179.235536,380449.258310,392011.544671,299924.867572,286914.767522,294396.764333,4.667525e+06,"POLYGON ((121.08178 14.63214, 121.08150 14.631..."
3,way/5105906,"Block 5 Lot 33 C M Recto Santo Nino, Marikina,...",16260.0,26614.3680,0.0,0.0,0.0,0.0,0.0,5.287131,...,479973.344935,445828.798471,432502.108422,389737.225059,401742.393690,312525.900464,303669.018655,313075.886428,4.806828e+06,"POLYGON ((121.11541 14.66517, 121.11576 14.663..."
4,way/5106137,"109 C.M. Recto St, Marikina, Metro Manila, Phi...",241.0,394.4688,0.0,0.0,0.0,0.0,0.0,0.123988,...,7532.154775,7021.139129,6851.704479,5931.394029,6190.946186,4649.227741,4362.122507,4496.389281,7.268944e+04,"POLYGON ((121.11880 14.66317, 121.11900 14.663..."


In [4]:
marikina_bounds_data = marikina_gdf[['id', 'admin_level', 'boundary', 'name', 'type', 'source:population', 'geometry']]

In [5]:
marikina_bounds_data['total_kwh'] = gdf[gdf['total_kwh'] >= 0]['total_kwh'].sum()

months = [f'avg_month_{x}' for x in range(1,13)]
hours = [f'avg_hour_{x}' for x in range(0,24)]

for month in months:
    marikina_bounds_data[month] = gdf[gdf[month] >= 0][month].mean()
for hour in hours:
    marikina_bounds_data[hour] = gdf[gdf[hour] >= 0][hour].mean()
marikina_bounds_data

/home/jupyter-sinagpala/.conda/envs/pipeline/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/jupyter-sinagpala/.conda/envs/pipeline/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/jupyter-sinagpala/.conda/envs/pipeline/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is tryin

,id,admin_level,boundary,name,type,source:population,geometry,total_kwh,avg_month_1,avg_month_2,...,avg_hour_14,avg_hour_15,avg_hour_16,avg_hour_17,avg_hour_18,avg_hour_19,avg_hour_20,avg_hour_21,avg_hour_22,avg_hour_23
0,relation/146949,6,administrative,Marikina,boundary,Philippines 2015 Census,"POLYGON ((121.09869 14.66569, 121.09839 14.664...",3.239323e+08,678.809254,741.913849,...,2.38996,1.654381,0.89561,0.237313,0.000225,0.0,0.0,0.0,0.0,0.0


In [6]:
marikina_bounds_data.to_json()
marikina_bounds_data.to_file("marikina_polygon_bounds.json")